In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

17464789/17464789 [==============================] - 2s 0us/step


In [5]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [6]:
word_index = keras.datasets.imdb.get_word_index()

1641221/1641221 [==============================] - 1s 1us/step


In [7]:
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}

In [8]:
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token

In [13]:
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [14]:
import tensorflow_datasets as tfds

In [16]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

In [18]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='C:\\Users\\User\\tensorflow_datasets\\imdb_reviews\\plain_text\\1.0.0',
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
        'train': 

In [19]:
train_size = info.splits["train"].num_examples

In [20]:
train_size

25000

In [21]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [22]:
from collections import Counter

In [23]:
vocabulary = Counter()

In [27]:
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [28]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [29]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

In [30]:
truncated_vocabulary

[b'<pad>',
 b'the',
 b'a',
 b'of',
 b'and',
 b'to',
 b'I',
 b'is',
 b'in',
 b'this',
 b'it',
 b'was',
 b'movie',
 b'that',
 b'The',
 b'film',
 b'with',
 b'for',
 b'as',
 b'on',
 b'but',
 b'have',
 b'This',
 b'one',
 b'not',
 b'be',
 b'are',
 b'you',
 b'an',
 b'at',
 b'about',
 b'by',
 b'all',
 b'his',
 b'so',
 b'like',
 b'from',
 b'who',
 b'has',
 b'It',
 b'good',
 b'my',
 b'just',
 b'very',
 b'out',
 b'or',
 b'story',
 b'some',
 b'time',
 b'had',
 b'he',
 b'they',
 b'really',
 b'me',
 b'when',
 b'what',
 b'first',
 b'movies',
 b'bad',
 b'see',
 b'seen',
 b'up',
 b'only',
 b'were',
 b"it's",
 b'would',
 b'more',
 b'made',
 b'great',
 b'can',
 b'been',
 b'i',
 b'her',
 b'no',
 b'A',
 b'which',
 b'even',
 b'films',
 b'there',
 b'ever',
 b'people',
 b'much',
 b'because',
 b'most',
 b'plot',
 b'if',
 b'than',
 b'acting',
 b'get',
 b'their',
 b'well',
 b'into',
 b'how',
 b'best',
 b'think',
 b'other',
 b'its',
 b"It's",
 b'saw',
 b'could',
 b'watch',
 b'many',
 b"don't",
 b'do',
 b'will',
 

In [31]:
words = tf.constant(truncated_vocabulary)

In [32]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [33]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

In [34]:
train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [38]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])

In [39]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [40]:
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 61s 74ms/step - loss: 0.6603 - accuracy: 0.5754
Epoch 2/5
782/782 [==============================] - 58s 74ms/step - loss: 0.4916 - accuracy: 0.7484
Epoch 3/5
782/782 [==============================] - 60s 76ms/step - loss: 0.3460 - accuracy: 0.8517
Epoch 4/5
782/782 [==============================] - 59s 75ms/step - loss: 0.2611 - accuracy: 0.8980
Epoch 5/5
782/782 [==============================] - 61s 78ms/step - loss: 0.1863 - accuracy: 0.9326


In [41]:
K = keras.backend

In [42]:
inputs=keras.layers.Input(shape=[None])

In [43]:
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)

In [44]:
model = keras.Model(inputs=[inputs], outputs=[outputs])

In [49]:
import tensorflow_hub as hub

In [50]:
model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1", dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [51]:
import os
TFHUB_CACHE_DIR = os.path.join(os.curdir, "my_tfhub_cache")
os.environ["TFHUB_CACHE_DIR"] = TFHUB_CACHE_DIR

In [52]:
for dirpath, dirnames, filenames in os.walk(TFHUB_CACHE_DIR):
    for filename in filenames:
        print(os.path.join(dirpath, filename))

.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe.descriptor.txt
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\saved_model.pb
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\assets\tokens.txt
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\variables\variables.data-00000-of-00001
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\variables\variables.index


In [53]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

In [54]:
train_size = info.splits["train"].num_examples
batch_size = 32

In [56]:
train_set = datasets["train"].batch(batch_size).prefetch(1)
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 3s 3ms/step - loss: 0.5456 - accuracy: 0.7264
Epoch 2/5
782/782 [==============================] - 2s 2ms/step - loss: 0.5145 - accuracy: 0.7471
Epoch 3/5
782/782 [==============================] - 2s 2ms/step - loss: 0.5091 - accuracy: 0.7504
Epoch 4/5
782/782 [==============================] - 2s 2ms/step - loss: 0.5054 - accuracy: 0.7528
Epoch 5/5
782/782 [==============================] - 2s 2ms/step - loss: 0.5026 - accuracy: 0.7540


In [57]:
import tensorflow_addons as tda

C:\Users\User\anaconda3\envs\mlbook\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.4.0 and strictly below 2.7.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [64]:
import numpy as np
ec_inputs = keras.layers.Input(shape=[None], dtype=np.int32)

dc_inputs = keras.layers.Input(shape=[None], dtype=np.int32) 

sequence_lenght = keras.layers.Input(shape=[], dtype=np.int32)

In [68]:
embeddings = keras.layers.Embedding(vocab_size, embed_size)

ec_embeddigs = embeddings(ec_inputs)
dc_embeddings = embeddings(dc_inputs)



In [69]:
ecoder = keras.layers.LSTM(512, return_state=True)

In [1]:
default_reber_grammar = [
    [("B", 1)],           # (state 0) =B=>(state 1)
    [("T", 2), ("P", 3)], # (state 1) =T=>(state 2) or =P=>(state 3)
    [("S", 2), ("X", 4)], # (state 2) =S=>(state 2) or =X=>(state 4)
    [("T", 3), ("V", 5)], # and so on...
    [("X", 3), ("S", 6)],
    [("P", 4), ("V", 6)],
    [("E", None)]] 

In [2]:
embedded_reber_grammar = [
    [("B", 1)],
    [("T", 2), ("P", 3)],
    [(default_reber_grammar, 4)],
    [(default_reber_grammar, 5)],
    [("T", 6)],
    [("P", 6)],
    [("E", None)]]

In [12]:
import numpy as np
def generate_string(grammar):
    state=0
    output=[]
    while state is not None:
        index = np.random.randint(len(grammar[state]))
        production, state = grammar[state][index]
        if isinstance(production, list):
            production = generate_string(grammar=production)
        output.append(production)
    return "".join(output)

In [13]:
for _ in range(25):
    print(generate_string(default_reber_grammar), end=" ")

BPTTTVPXVVE BTXSE BTSSSSSXXTTTVPXVVE BPTVPSE BTXXTTVVE BPVPSE BPTVVE BTXXVPXTVPXVPSE BPVVE BTXSE BTXXTVPXTVVE BTSXSE BPVVE BTSSXSE BTXXVVE BTSXXVVE BTSSSXXTTTTVVE BPTTTVVE BPTVVE BTXSE BPTVPXVPSE BTSXSE BTXSE BPTTTTTTVPXVPXTTVVE BPTTTVPXVPXTTVVE 

In [14]:
POSSIBLE_CHARS = "BEPSTVX"
def generate_corrupted_string(grammar, chars=POSSIBLE_CHARS):
    good_string = generate_string(grammar)
    index = np.random.randint(len(good_string))
    good_char = good_string[index]
    bad_char = np.random.choice(sorted(set(chars) - set(good_char)))
    return good_string[:index] + bad_char + good_string[index + 1:]

In [15]:
for _ in range(25):
    print(generate_corrupted_string(embedded_reber_grammar), end=" ")

XPBPVVEPE BTBTXXTTTTTTXPSETE BPBTXXTTTBVVEPE BPBPVPXVPXVESEPE BPVTSXSEPE BPETXSEPE BPBTXSESE ETBPVVETE BTBTXXTVPEVVETE BTBTSXXVPXTVTSETE STBTXSETE BTSTXSETE BPVTSXXVVEPE BPBTSSXSEEE BPBTXVEPE BPBTSSXSPPE BPBTSSXXTTVPSETE BPBTVXSEPE STBPTVPXVPSETE BPXTSSXSEPE BTBVXXVPXTTTTTTTTVPSETE BTBBVPSETE BTBPTTTTVVTTE BTBTXXTTTTTVPXVVEXE BPBTSSSSSSEPE 

In [16]:
def string_to_ids(s, chars=POSSIBLE_CHARS):
    return [chars.index(c) for c in s]

In [17]:
string_to_ids("BTTTXXVVETE")

[0, 4, 4, 4, 6, 6, 5, 5, 1, 4, 1]

In [20]:
import tensorflow as tf
def generate_dataset(size):
    good_strings = [string_to_ids(generate_string(embedded_reber_grammar))
                    for _ in range(size // 2)]
    bad_strings = [string_to_ids(generate_corrupted_string(embedded_reber_grammar))
                   for _ in range(size - size // 2)]
    all_strings = good_strings + bad_strings
    X = tf.ragged.constant(all_strings, ragged_rank=1)
    y = np.array([[1.] for _ in range(len(good_strings))] +
                 [[0.] for _ in range(len(bad_strings))])
    return X, y

In [26]:
X_train, y_train = generate_dataset(10000)
X_valid, y_valid = generate_dataset(2000)

In [27]:
X_train[0]

<tf.Tensor: shape=(15,), dtype=int32, numpy=array([0, 4, 0, 2, 4, 5, 2, 6, 4, 5, 2, 3, 1, 4, 1])>

In [28]:
y_train[0]

array([1.])

In [30]:
embedding_size = 5
from tensorflow import keras
model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=[None], dtype=tf.int32, ragged=True),
    keras.layers.Embedding(input_dim=len(POSSIBLE_CHARS), output_dim=embedding_size),
    keras.layers.GRU(30),
    keras.layers.Dense(1, activation="sigmoid")
])


In [31]:
optimizer = keras.optimizers.SGD(learning_rate=0.02, momentum = 0.95, nesterov=True)

In [32]:
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [48]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
313/313 [==============================] - 2s 6ms/step - loss: 0.3365 - accuracy: 0.8646 - val_loss: 0.4592 - val_accuracy: 0.7175
Epoch 2/20
313/313 [==============================] - 2s 6ms/step - loss: 0.2053 - accuracy: 0.9321 - val_loss: 0.1129 - val_accuracy: 0.9735
Epoch 3/20
313/313 [==============================] - 2s 6ms/step - loss: 0.1182 - accuracy: 0.9650 - val_loss: 0.0844 - val_accuracy: 0.9805
Epoch 4/20
313/313 [==============================] - 2s 6ms/step - loss: 0.0918 - accuracy: 0.9774 - val_loss: 0.1764 - val_accuracy: 0.9480
Epoch 5/20
313/313 [==============================] - 2s 6ms/step - loss: 0.1926 - accuracy: 0.9303 - val_loss: 0.1040 - val_accuracy: 0.9745
Epoch 6/20
313/313 [==============================] - 2s 6ms/step - loss: 0.0885 - accuracy: 0.9786 - val_loss: 0.0849 - val_accuracy: 0.9790
Epoch 7/20
313/313 [==============================] - 2s 6ms/step - loss: 0.0800 - accuracy: 0.9805 - val_loss: 0.0704 - val_accuracy: 0.9815
Epoch 

In [105]:
test_strings = ["BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE",
                "BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE"]

In [106]:
X_test = tf.ragged.constant([string_to_ids(s) for s in test_strings], ragged_rank=1)

In [107]:
y_proba = model.predict(X_test)

1/1 [==============================] - 0s 10ms/step


In [108]:
y_proba

array([[0.001542 ],
       [0.9998921]], dtype=float32)

In [116]:
for index, string in enumerate(test_strings):
    print("{}: {:.2f}%".format(string, 100 * y_proba[index][0]))

BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE: 0.15%
BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE: 99.99%


In [52]:
MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

import calendar
from datetime import date
import random

def generate_random_date():
    # Gera um ano aleatório
    year = random.randint(1900, 2024)
    
    # Gera um mês aleatório
    month = random.randint(1, 12)
    
    # Obtém o número de dias do mês e ano gerado
    _, num_days = calendar.monthrange(year, month)
    
    # Gera um dia aleatório dentro do intervalo válido para o mês
    day = random.randint(1, num_days)
    
    # Cria a data
    random_date = date(year, month, day)
    
    # Converte a data para o formato desejado
    date_str = random_date.strftime("%B %d, %Y")
    return date_str

# Teste
for _ in range(5):
    print(generate_random_date())


test = generate_random_date()

print(test)

from datetime import datetime
def convert_format(date_str):
    date_obj = datetime.strptime(date_str, '%B %d, %Y')
    return date_obj.strftime('%Y-%m-%d')

examples = []
for _ in range(10):
    dates = generate_random_date()
    print(dates)
    new_date = convert_format(dates)
    examples.append([dates, new_date])

examples

def create_dataset(n_samples):
    X = []
    y = []
    for _ in range(n_samples):
        date_str = generate_random_date()
        X.append(date_str)
        target = convert_format(date_str)
        y.append(target)
    return X, y



n_samples = 100
test1, test1_y = create_dataset(n_samples)
test1

vocab = []
for month in MONTHS:
    for char in month:
        if char not in vocab:
            vocab.append(char)
print(vocab)

for i in range(10):
    vocab.append(str(i))

INPUT_CHARS = "0123456789-ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz ,"
from tensorflow import keras
def date_str_to_ids(date_str, chars=INPUT_CHARS):
    return [chars.index(c) for c in date_str]

# Exemplo com uma string de data
date_str = "April 22, 2019"
date_ids = date_str_to_ids(date_str, INPUT_CHARS)
print(date_ids)


date_str = "April 21, 2019"
date_ids = date_str_to_ids(date_str, INPUT_CHARS)
print(date_ids)


X_train, y_train = create_dataset(10000)
X_test, y_test = create_dataset(2000)
X_val, y_val = create_dataset(2000)

import numpy as np
y_train = np.array(y_train)
max_output_length = len(y_train[0])

import tensorflow as tf

# Corrigir: transformar para listas de IDs
X_train_ids = [date_str_to_ids(s, INPUT_CHARS) for s in X_train]
X_val_ids   = [date_str_to_ids(s, INPUT_CHARS) for s in X_val]
y_train_ids = [date_str_to_ids(s, INPUT_CHARS) for s in y_train]
y_val_ids   = [date_str_to_ids(s, INPUT_CHARS) for s in y_val]

# Criar RaggedTensor para entrada
X_train = tf.ragged.constant(X_train_ids, dtype=tf.int32)
X_val   = tf.ragged.constant(X_val_ids, dtype=tf.int32)

# Saída como tensor denso (mesmo tamanho sempre no formato YYYY-MM-DD)
y_train = tf.constant(y_train_ids, dtype=tf.int32)
y_val   = tf.constant(y_val_ids, dtype=tf.int32)

encoder = keras.models.Sequential([
    keras.layers.Input(shape=(None,), ragged=True, dtype="int32"),
    keras.layers.Embedding(input_dim=len(INPUT_CHARS) + 1, output_dim=100),
    keras.layers.LSTM(128)
])

decoder = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.Dense(len(INPUT_CHARS) + 1, activation="softmax")
])

model = keras.models.Sequential([
    encoder,
    keras.layers.RepeatVector(max_output_length),
    decoder
])
optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

November 25, 1916
March 27, 2020
April 01, 1953
September 21, 1928
May 14, 1942
October 25, 1952
February 22, 2022
June 08, 1919
April 15, 1982
August 11, 2015
April 08, 2011
August 01, 2019
February 11, 2002
April 17, 1949
February 15, 1908
December 31, 1996
['J', 'a', 'n', 'u', 'r', 'y', 'F', 'e', 'b', 'M', 'c', 'h', 'A', 'p', 'i', 'l', 'g', 's', 't', 'S', 'm', 'O', 'o', 'N', 'v', 'D']
[11, 52, 54, 45, 48, 63, 2, 2, 64, 63, 2, 0, 1, 9]
[11, 52, 54, 45, 48, 63, 2, 1, 64, 63, 2, 0, 1, 9]


In [53]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Epoch 1/20


C:\Users\User\anaconda3\envs\mlbook\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_12/sequential_10/lstm_7/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_12/sequential_10/lstm_7/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 100), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_12/sequential_10/lstm_7/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


313/313 [==============================] - 10s 21ms/step - loss: 1.7737 - accuracy: 0.4021 - val_loss: 1.2421 - val_accuracy: 0.5180
Epoch 2/20
313/313 [==============================] - 6s 20ms/step - loss: 1.0835 - accuracy: 0.6056 - val_loss: 0.8953 - val_accuracy: 0.6711
Epoch 3/20
313/313 [==============================] - 6s 20ms/step - loss: 0.6861 - accuracy: 0.7673 - val_loss: 0.4850 - val_accuracy: 0.8414
Epoch 4/20
313/313 [==============================] - 6s 20ms/step - loss: 0.3356 - accuracy: 0.9088 - val_loss: 0.2136 - val_accuracy: 0.9498
Epoch 5/20
313/313 [==============================] - 6s 20ms/step - loss: 0.1202 - accuracy: 0.9840 - val_loss: 0.0638 - val_accuracy: 0.9979
Epoch 6/20
313/313 [==============================] - 6s 19ms/step - loss: 0.0391 - accuracy: 0.9996 - val_loss: 0.0243 - val_accuracy: 1.0000
Epoch 7/20
313/313 [==============================] - 6s 20ms/step - loss: 0.0168 - accuracy: 1.0000 - val_loss: 0.0124 - val_accuracy: 1.0000
Epoch 8/2

In [69]:
X_new = date_str_to_ids("September 17, 2009", INPUT_CHARS)
X_new_ragged = tf.ragged.constant([X_new], dtype=tf.int32)


pred = model.predict(X_new_ragged)

1/1 [==============================] - 1s 516ms/step


In [70]:
print(pred)

[[[6.06561662e-06 2.65691033e-05 9.99958277e-01 1.20932066e-06
   1.89052869e-07 2.67575351e-10 1.53370691e-10 2.22395222e-08
   9.67464331e-08 8.31002112e-08 2.19541832e-07 1.14669433e-07
   2.28936884e-07 9.69453708e-08 2.13713619e-07 1.58562855e-07
   1.19060417e-07 9.03764743e-08 1.25418723e-07 1.05091594e-07
   9.72747003e-08 8.40092440e-08 1.45499087e-07 1.36460883e-07
   6.72591369e-08 1.94589944e-07 1.34151037e-07 7.36523376e-08
   1.79632949e-07 1.73990529e-07 6.45965486e-08 7.46202389e-08
   1.43877628e-07 1.53673696e-07 1.62878990e-07 1.50736440e-07
   1.46256028e-07 2.25131686e-07 1.64559935e-07 5.63306060e-08
   7.08515770e-08 1.12026143e-07 3.04462617e-07 2.60902937e-07
   1.15477462e-07 1.41858564e-07 1.92993141e-07 7.83968304e-08
   1.16081651e-07 1.94005878e-07 1.57486241e-07 1.34645020e-07
   6.47797691e-08 1.97530284e-07 4.86967338e-08 2.12709125e-07
   1.26040419e-07 1.06016692e-07 9.88551818e-08 7.23001392e-08
   7.15687136e-08 1.00324442e-07 1.89590907e-07 9.97680

In [71]:
ids = np.argmax(pred, axis=-1)

# Função para converter IDs para string
def ids_to_date_strs(ids_array, chars=INPUT_CHARS):
    return ["".join([chars[i] for i in row]) for row in ids_array]

# Converta para string e imprima
for date_str in ids_to_date_strs(ids):
    print(date_str)

2009-09-17
